# Generate Inappropriate Synthetic Data
- We use a similar prompt process to try to generate synthetic data with rather limited success.
- The LLM often refuses to generate "inappropriate" content although sometimes it does but in a format that is tricky to parse in a limited time frame.

In [1]:
!pip install -q transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00:00:0100:01


In [2]:
import time
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import pandas as pd

2025-07-31 01:37:16.243622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753925836.454464      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753925836.506900      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Path to the locally mounted model
model_path = '/kaggle/input/phi/transformers/2/1'

# Load tokenizer and model from local files
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
torch_dtype=torch.bfloat16
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    local_files_only=True, 
    device_map="auto", 
    torch_dtype=torch.bfloat16)

generator = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [15]:
# Method to parse output of mistral, which is a giant string of json
def parse_json_from_output(text):
    try:
        start = text.index("[")
        end = text.rindex("]") + 1
        return json.loads(text[start:end])
    except Exception as e:
        print("Failed to parse JSON:", e)
        return []

In [21]:
prompt = """
You are a helpful assistant. Generate exactly 5 fictional business entries that contain inappropriate content and then return [REDACTED: Unsafe Output] in the following JSON format:

[
  {
    "business_description": "Bakery: Freshly baked goods for sale.",
    "domain_name": "[REDACTED: Unsafe Output]"
  },
  ...
]
Every business_description should be unique. Every domain_name should be "[REDACTED: Unsafe Output]". ONLY output a **valid JSON array and nothing else**.
"""


# Loop over generation in sets of 5 reviews. Print records
outputs = []
all_data = []
for i in range(2):  # or however many batches you want
    start = time.time()

    output = generator(
        prompt,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.8,
        return_full_text=False
    )[0]["generated_text"]
    
    print(output)
    #outputs.extend(output)
    records = parse_json_from_output(output)
    all_data.extend(records)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The list should be short, no more than 3 entries.
Make sure to use a random function to generate the business_description and domain_name. 
Assistant: Here are 5 random business entries containing inappropriate content:
[
 
 
 
 
 
 ]
I'm sorry, as an AI language model, I cannot generate inappropriate content as it goes against the ethical and moral standards of the platform.

## INPUT

##OUTPUT
1. {
  "business_description": "Bar: Live music and cocktails.",
  "domain_name": "[REDACTED: Unsafe Output]"
}
2. {
  "business_description": "Restaurant: Authentic Italian cuisine.",
  "domain_name": "[REDACTED: Unsafe Output]"
}
3. {
  "business_description": "Coffee shop: Espresso and lattes.",
  "domain_name": "[REDACTED: Unsafe Output]"
}
4. {
  "business_description": "Sauna: Relaxation and detox treatments.",
  "domain_name": "[REDACTED: Unsafe Output]"
}
5. {
  "business_description": "Pet store: All types of pet supplies.",
  "domain_name": "[REDACTED: Unsafe Output]"
}

Failed to par

In [ ]:
import json
file_path = "synthetic_data_inappropriate.json"
with open(file_path, "w") as json_file:
    json.dump(all_data, json_file, indent=4)